#. 🏦 Banking Service Agent 💵

**What's about:** Banking Customer Service Agent with RAG and Sqlite toolsfor products and policy related questions.

Dev by Jeison Robles.

In [35]:
from IPython.display import display, HTML

def show_agent_banner():
    display(HTML("""
    <div style="
        margin: 10px auto 20px auto;
        padding: 18px 22px;
        max-width: 720px;
        border-radius: 18px;
        background: linear-gradient(135deg, #0f172a, #1e293b, #020617);
        color: #e5e7eb;
        font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto;
        box-shadow: 0 18px 40px rgba(0,0,0,.45);
    ">
        <div style="display:flex; align-items:center; gap:16px;">
            <div style="
                font-size:78px;
                line-height:1;
            ">
                🐙
            </div>
            <div>
                <div style="font-size:20px; font-weight:700;">
                    Banking Service Agent
                </div>
                <div style="font-size:13px; opacity:.85;">
                    Policy-aware • RAG • SQLite • Safe-by-default
                </div>
            </div>
        </div>

        <div style="
            margin-top:12px;
            font-size:14px;
            line-height:1.5;
            opacity:.9;
        ">
            Ask general policy questions freely.<br>
            Provide a <b>Customer ID</b> only when you want personal product information.
        </div>

        <div style="
            margin-top:14px;
            padding-top:10px;
            border-top:1px solid rgba(255,255,255,.1);
            font-size:12px;
            opacity:.7;
        ">
            Agent status: <span style="color:#22d3ee;">ONLINE</span>
        </div>
    </div>
    """))

show_agent_banner()

In [17]:
%pip install -q openai qdrant-client docling fastembed jupyter-chat-widget

In [18]:
from jupyter_chat_widget import ChatUI
from time import sleep

chat = ChatUI()

def answers(text):
  chat.rewrite("Thinking...")
  sleep(3)
  chat.rewrite("")
  chat.append("Hello "+ text)

chat.connect(answers)


Output()

Output()

Text(value='', description='user: ')

# Prep Functions

In [19]:
import openai
import os
from google.colab import userdata


def get_oai_client():
    """
        Creates the OpenAI client.
    """
    os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
    client = openai.OpenAI()
    messages = [{"role": "system",
                 "content": """
                 You are a helpful banking assistant that will provide easy to understand answers about the bank policy or related to customers products,
                 If the user asks about my products/accounts and provides a customer_id, call get_customer_name and get_customer_products.
                 When customer products are available, always respond using this structure:

                 1) Greeting (with customer name if known)
                 2) Product summary (bullets). Include product_type, nickname, currency, balance, credit_limit, apr.
                 3) What the negative balance means (if any), in plain language.
                 4) What actions are allowed according to policy (cite from snippets).
                 Then summarize what products they have and answer using the policy with the search tool.
                 Output in HTML."""}]
    return client, messages


def query_llm(ui, client, messages):
    """
        Query the LLM and returns a stream handle to the response.
    """
    ui.rewrite("[Generating]")
    stream = client.chat.completions.create(
        model="gpt-5.2",  # Or another suitable model
        messages=messages, # Use the message history
        stream=True, # Enable streaming
    )
    return stream


def display_response(ui, stream):
    """
        Display the LLM's response in the UI one token at a time.
    """
    ui.rewrite("")
    complete_response = ""
    for chunk in stream:
        if (content := chunk.choices[0].delta.content) and content.strip():
            complete_response += content
            ui.append(content)
    return complete_response


# Import Policy

In [20]:
import os
import requests

urls_and_filenames = [
    (
        "https://raw.githubusercontent.com/JeisonRobles/Banking-Service-Agent/main/Bank_Policy_1",
        "Bank_Policy_1.md"
    ),
]

os.makedirs("documents", exist_ok=True)

for url, filename in urls_and_filenames:
    response = requests.get(url)
    response.raise_for_status()
    with open(os.path.join("documents", filename), "wb") as f:
        f.write(response.content)

___
# Documents processing

In [21]:
from docling.document_converter import DocumentConverter
from docling.chunking import HybridChunker
from docling.datamodel.base_models import InputFormat
from docling.document_converter import DocumentConverter
from uuid import uuid4
from qdrant_client import QdrantClient, models


def ingest_documents(ui, paths):
    print("Ingesting documents...")
    print(paths)
    # Prepare the vectordb and embedder
    vdb = QdrantClient(location=":memory:")
    dense_model = "sentence-transformers/all-MiniLM-L6-v2"
    vdb.set_model(dense_model)
    collection_name = "documents"
    vdb.create_collection(
        collection_name=collection_name,
        vectors_config=vdb.get_fastembed_vector_params(),
    )
    points = []
    for path in paths:
        print(path)

    print(f"processing path: {paths}")

    ui.rewrite(f"[Parsing {paths}...]")


    for path in paths:

        print(f"processing path: {path}")

        ui.rewrite(f"[Parsing {path}...]")

        # Parse the document with docling
        doc = DocumentConverter().convert(source=path).document

        # Chunk the document
        chunker = HybridChunker()
        chunk_iter = chunker.chunk(dl_doc=doc)

        # Enrich the chunks and build Qdrant points
        for chunk in chunk_iter:
            enriched_text = chunker.contextualize(chunk=chunk)
            meta = chunk.meta.export_json_dict()
            points.append(
                models.PointStruct(
                    id=uuid4().hex,
                    payload=meta | {"document": enriched_text},
                    vector={
                        # FastEmbed uses named vector fields derived from the model names
                        vdb.get_vector_field_name(): models.Document(text=enriched_text, model=dense_model),
                    },
                )
            )
    # Upload (embeddings happen internally because we used models.Document)
    vdb.upload_points(collection_name=collection_name, points=points, batch_size=64, wait=True)
    ui.rewrite(f"All documents were processed. I'm ready!")
    return vdb

def add_context(ui, vdb, query):
    """
        Queries the vector database for relevant context snippets
        and adds them to the LLM's context.
    """

    print("Adding context...")
    ui.rewrite("[Searching]")
    samples = vdb.query(
        collection_name="documents",
        query_text=query,
        limit=10,
    )
    ui.rewrite(f"[Found {len(samples)} relevant snippets]")
    sleep(1)
    return {
        "role": "user",
        "content": f"Relevant snippets from the document: {'\n\n'.join(s.document for s in samples)}"
    }

## 💾 SQL Local DB

Lets implement an small SQLite db for customers patterns


In [22]:
import sqlite3
from pathlib import Path

DB_PATH = Path("bank_demo.sqlite")

def init_customer_db(db_path: Path = DB_PATH):
    conn = sqlite3.connect(str(db_path))
    conn.execute("PRAGMA foreign_keys = ON;")
    cur = conn.cursor()

    # 1) Customers
    cur.execute("""
    CREATE TABLE IF NOT EXISTS customers (
      customer_id TEXT PRIMARY KEY,
      full_name   TEXT NOT NULL
    )
    """)

    # 2) Products (linked to customers)
    cur.execute("""
    CREATE TABLE IF NOT EXISTS products (
      product_id   TEXT PRIMARY KEY,
      customer_id  TEXT NOT NULL,
      product_type TEXT NOT NULL,
      nickname     TEXT,
      currency     TEXT,
      balance      REAL,
      credit_limit REAL,
      apr          REAL,
      FOREIGN KEY(customer_id) REFERENCES customers(customer_id)
    )
    """)

    # Seed customers (idempotent)
    cur.executemany(
        "INSERT OR IGNORE INTO customers (customer_id, full_name) VALUES (?, ?)",
        [
            ("CUST-1001", "Alicia Mora"),
            ("CUST-2002", "Diego Hernández"),
            ("CUST-3003", "María José Vargas"),
        ],
    )

    # Seed products (idempotent)
    cur.executemany(
        """
        INSERT OR IGNORE INTO products
        (product_id, customer_id, product_type, nickname, currency, balance, credit_limit, apr)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        """,
        [
            ("P-CHK-01", "CUST-1001", "checking",       "Daily Checking",   "USD",  1250.75,   None,     None),
            ("P-CC-01",  "CUST-1001", "credit_card",    "Travel Card",      "USD",  -230.10,  5000.00,  0.279),
            ("P-SAV-01", "CUST-2002", "savings",        "Business Savings", "USD", 18250.00,   None,     None),
            ("P-LOC-01", "CUST-2002", "line_of_credit", "Working Capital",  "USD", -1200.00, 25000.00,  0.185),
        ],
    )

    conn.commit()
    conn.close()
    return str(db_path.resolve())

DB_RESOLVED = init_customer_db()
print(f"SQLite demo DB ready at: {DB_RESOLVED}")

SQLite demo DB ready at: /content/bank_demo.sqlite


In [23]:

# --- Tool: lookup customer name (used only for "my products"/personal info requests) ---
import sqlite3
from typing import Dict, Any
import json

def get_customer_name(customer_id: str) -> Dict[str, Any]:
    """Return customer name for greeting; no products yet."""
    conn = sqlite3.connect(str(DB_PATH))
    cur = conn.cursor()
    cur.execute("SELECT full_name FROM customers WHERE customer_id = ?", (customer_id,))
    row = cur.fetchone()
    conn.close()
    if not row:
        return {"found": False, "customer_id": customer_id, "full_name": None}
    return {"found": True, "customer_id": customer_id, "full_name": row[0]}

# Tool implementations registry (keep/extend your existing one if present)
try:
    tool_implementations
except NameError:
    tool_implementations = {}


#Define tool implementations:
tool_implementations["get_customer_name"] = lambda ui, vdb, customer_id: {
    "role": "tool",
    "name": "get_customer_name",
    "content": json.dumps(get_customer_name(customer_id), ensure_ascii=False),
}



#=========================================================
# Defining get_customer_products tool
#=========================================================
import sqlite3
from typing import Dict, Any, List
import json

def get_customer_products(customer_id: str) -> Dict[str, Any]:
    conn = sqlite3.connect(str(DB_PATH))
    cur = conn.cursor()
    cur.execute("""
        SELECT product_id, product_type, nickname, currency, balance, credit_limit, apr
        FROM products
        WHERE customer_id = ?
        ORDER BY product_type
    """, (customer_id,))
    rows = cur.fetchall()
    conn.close()

    products: List[Dict[str, Any]] = []
    for r in rows:
        products.append({
            "product_id": r[0],
            "product_type": r[1],
            "nickname": r[2],
            "currency": r[3],
            "balance": r[4],
            "credit_limit": r[5],
            "apr": r[6],
        })

    return {
        "customer_id": customer_id,
        "count": len(products),
        "products": products
    }

# register tool implementation (DO NOT overwrite tool_implementations)
tool_implementations.update({
    "get_customer_products": lambda ui, vdb, customer_id: {
        "role": "tool",
        "name": "get_customer_products",
        "content": json.dumps(get_customer_products(customer_id), ensure_ascii=False),
    }
})


___
# Retrieval as a tool

In [24]:
import json

def _invoke_tool(ui, vdb, tool_call):
    tool_name = tool_call.function.name
    ui.rewrite(f"Calling tool {tool_name}")

    tool_args = json.loads(tool_call.function.arguments or "{}")

    tool_impl = tool_implementations[tool_name]
    output = tool_impl(ui, vdb, **tool_args)  # returns {"role":"tool","name":...,"content":...}

    # IMPORTANT: tool message MUST include tool_call_id
    tool_msg = {
        "role": "tool",
        "tool_call_id": tool_call.id,
        "content": output["content"],
    }
    return tool_msg


def query_llm_with_tools(ui, client, vdb, messages, tools, max_rounds=5):
    """
    Robust minimal loop:
    - Keep resolving tool calls until the model returns a normal assistant message (no tool_calls)
    - Then stream the final answer
    """
    for _ in range(max_rounds):
        ui.rewrite("[Thinking...]")

        resp = client.chat.completions.create(
            model="gpt-5.2",
            messages=messages,
            tools=tools,
            tool_choice="auto",
            stream=False,
        )

        assistant_msg = resp.choices[0].message
        tool_calls = getattr(assistant_msg, "tool_calls", None)

        # If the model is NOT requesting tools, stream final answer
        if not tool_calls:
            ui.rewrite("[Generating]")
            stream = client.chat.completions.create(
                model="gpt-5.2",
                messages=messages,
                stream=True,
            )
            return stream

        # Append assistant message WITH tool_calls (required)
        messages.append({
            "role": "assistant",
            "content": assistant_msg.content,
            "tool_calls": [tc.model_dump() for tc in tool_calls],
        })

        # Execute tools and append tool results
        for tc in tool_calls:
            tool_msg = _invoke_tool(ui, vdb, tc)
            messages.append(tool_msg)

    # If we hit max rounds, fail safely
    return client.chat.completions.create(
        model="gpt-5.2",
        messages=messages + [{"role": "user", "content": "Please answer without calling more tools."}],
        stream=True,
    )

In [36]:
# keep existing tools and add/override entries
tool_implementations.update({
    "search": add_context,
    "get_customer_name": lambda ui, vdb, customer_id: {
        "role": "tool",
        "name": "get_customer_name",
        "content": json.dumps(get_customer_name(customer_id), ensure_ascii=False),
    }
})


#=========================================================
# Set search schema
#=========================================================
search = {
    "type": "function",
    "function": {
        "name": "search",
        "description": "Use this tool when the user is asking you something you don't know. You should use this tool very often and you can call it many times in a row if necessary.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description":
                        "The query to retrieve the information from the document store using pure embedding similarity search",
                },
            },
            "required": ["query"],
        },
    }
}


#=========================================================
# Set get_customer_name schema
#=========================================================
get_customer_name_tool = {
  "type": "function",
  "function": {
    "name": "get_customer_name",
    "description": "Look up the customer's full name by customer_id for a personalized greeting. Use only when the user asks about their own products/accounts and provides a customer_id.",
    "parameters": {
      "type": "object",
      "properties": {
        "customer_id": {
          "type": "string",
          "description": "Customer ID in format CUST-#### (e.g., CUST-2002)."
        }
      },
      "required": ["customer_id"]
    }
  }
}

#=========================================================
# Set get_customer_products schema
#=========================================================
get_customer_products_tool = {
  "type": "function",
  "function": {
    "name": "get_customer_products",
    "description": "Fetch the list of products for a customer_id. Use only when the user asks about their own products/accounts and provides a customer_id.",
    "parameters": {
      "type": "object",
      "properties": {
        "customer_id": {
          "type": "string",
          "description": "Customer ID in format CUST-#### (e.g., CUST-2002)."
        }
      },
      "required": ["customer_id"]
    }
  }
}



def retrieval_as_tool(paths):
    show_agent_banner()
    ui = ChatUI()
    vdb = ingest_documents(ui, paths)
    client, messages = get_oai_client()

    def _retrieval_as_tool(query):
        messages.append({"role": "user", "content": query})
        #stream = query_llm_with_tools(ui, client, vdb, messages, [search])
        #stream = query_llm_with_tools(ui, client, vdb, messages, [search, get_customer_name_tool]) ## Calling the model with both tools
        stream = query_llm_with_tools(
            ui, client, vdb, messages,
            [search, get_customer_name_tool, get_customer_products_tool]
        )
        response = display_response(ui, stream)
        messages.append({"role": "assistant", "content": response})

    ui.connect(lambda query: _retrieval_as_tool(query))



In [37]:
retrieval_as_tool(["/content/documents/Bank_Policy_1.md"])

Output()

Output()

Text(value='', description='user: ')

Ingesting documents...
['/content/documents/Bank_Policy_1.md']
/content/documents/Bank_Policy_1.md
processing path: ['/content/documents/Bank_Policy_1.md']
processing path: /content/documents/Bank_Policy_1.md
Adding context...
Adding context...
